In [1]:
! pip install opacus
! pip install medmnist

# Task 1

In [1]:
import warnings
warnings.simplefilter("ignore")
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus import PrivacyEngine
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
from opacus.validators import ModuleValidator
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Task 1a\)

In [2]:
### Dataset preparations
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)


# CIFAR-10 dataset loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

train_dataset = datasets.CIFAR10(root='./data/cifar10', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data/cifar10', train=False, download=True, transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

### Hyperparameters
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 20
LR = 1e-3
BATCH_SIZE = 256
MAX_PHYSICAL_BATCH_SIZE = 32

Files already downloaded and verified
Files already downloaded and verified


## Define methods

In [3]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [5]:
##############################################################################################
# Image classifier WITH differential privacy: Train and test method
##############################################################################################

def train_dp(model_dp, train_loader_dp, optimizer_dp, epoch, device, privacy_engine):
    model_dp.train()
    criterion_dp = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []
        
    with BatchMemoryManager(
        data_loader=train_loader_dp, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer_dp
    ) as memory_safe_data_loader:
        for i, (images, target) in enumerate(memory_safe_data_loader):
            optimizer_dp.zero_grad()
            images = images.to(device)
            target = target.to(device)
            
            output = model_dp(images)
            loss = criterion_dp(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer_dp.step()

            if (i + 1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )


def test_dp(model_dp, test_loader_dp, device):
    model_dp.eval()
    criterion_dp = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader_dp:
            images = images.to(device)
            target = target.to(device)

            output = model_dp(images)
            loss = criterion_dp(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)


    print(
        f"\tTest set: "
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)


def eval_dp(model_dp, val_loader_dp, device):
    model_dp.eval()
    criterion_dp = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in val_loader_dp:
            images = images.to(device)
            target = target.to(device)

            output = model_dp(images)
            loss = criterion_dp(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)


    print(
        f"\tValidation set: "
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [6]:
##############################################################################################
# Image classifier WITHOUT differential privacy: Train and test method
##############################################################################################

def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    for i, (images, target) in enumerate(train_loader):
        optimizer.zero_grad()
        images = images.to(device)
        target = target.to(device)
        
        output = model(images)
        loss = criterion(output, target)

        preds = np.argmax(output.detach().cpu().numpy(), axis=1)
        labels = target.detach().cpu().numpy()

        acc = accuracy(preds, labels)

        losses.append(loss.item())
        top1_acc.append(acc)

        loss.backward()
        optimizer.step()

        if (i + 1) % 50 == 0:
            print(
                f"\tTrain Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
                f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
            )


def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)


    print(
        f"\tTest set: "
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)


def eval(model, val_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in val_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)


    print(
        f"\tValidation set: "
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

## With differential privacy

### CIFAR-10

In [7]:

##############################################################################################
# CIFAR-10 classification WITH differential privacy: Model, Optimizer, DataLoader
##############################################################################################

train_loader_dp = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader_dp = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader_dp = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_cifar_dp = models.resnet18(num_classes=10)
model_cifar_dp = ModuleValidator.fix(model_cifar_dp)
ModuleValidator.validate(model_cifar_dp, strict=False)

model_cifar_dp = model_cifar_dp.to(device)

optimizer_dp = optim.Adam(model_cifar_dp.parameters(), lr=LR)


print(
    f"Before make_private(). "
    f"Model:{type(model_cifar_dp)}, Optimizer:{type(optimizer_dp)}, DataLoader:{type(train_loader_dp)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_cifar_dp, optimizer_dp, train_loader_dp = privacy_engine.make_private_with_epsilon(
    module=model_cifar_dp,
    optimizer=optimizer_dp,
    data_loader=train_loader_dp,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_dp.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_cifar_dp)}, Optimizer:{type(optimizer_dp)}, DataLoader:{type(train_loader_dp)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_cifar_dp, train_loader_dp, optimizer_dp, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_cifar_dp, val_loader_dp, device)

top1_acc_cifar_dp = test_dp(model_cifar_dp, test_loader_dp, device)
print("CIFAR DP Top1 Acc: ", top1_acc_cifar_dp)
torch.save(model_cifar_dp, 'cifar-10_dp.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3569221496582031 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.204579 Acc@1: 18.720171 (ε = 9.83, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.091480 Acc@1: 23.785138 (ε = 11.36, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.010082 Acc@1: 27.175569 (ε = 12.48, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.956804 Acc@1: 29.568998 (ε = 13.35, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.923658 Acc@1: 31.164948 (ε = 14.15, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.892618 Acc@1: 32.519815 (ε = 14.84, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [01:18<24:52, 78.57s/epoch]

	Train Epoch: 2 	Loss: 1.756921 Acc@1: 40.545373 (ε = 15.90, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.748925 Acc@1: 40.633224 (ε = 16.49, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.739691 Acc@1: 41.197516 (ε = 17.05, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.738532 Acc@1: 41.544515 (ε = 17.56, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.735312 Acc@1: 41.826725 (ε = 18.08, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.736550 Acc@1: 42.042757 (ε = 18.57, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [26:50<01:28, 88.14s/epoch]

	Train Epoch: 20 	Loss: 1.614281 Acc@1: 60.100388 (ε = 48.87, δ = 1e-05)
	Train Epoch: 20 	Loss: 1.607946 Acc@1: 60.305162 (ε = 49.07, δ = 1e-05)
	Train Epoch: 20 	Loss: 1.622305 Acc@1: 60.221675 (ε = 49.27, δ = 1e-05)
	Train Epoch: 20 	Loss: 1.605686 Acc@1: 60.364717 (ε = 49.47, δ = 1e-05)
	Train Epoch: 20 	Loss: 1.602880 Acc@1: 60.339347 (ε = 49.67, δ = 1e-05)
	Train Epoch: 20 	Loss: 1.605178 Acc@1: 60.250614 (ε = 49.87, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [28:19<00:00, 84.96s/epoch]

	Validation set: Loss: 1.817502 Acc: 57.900391 


	Test set: Loss: 1.846840 Acc: 56.806641 
CIFAR DP Top1 Acc:  0.56806640625


## Without differential privacy

### CIFAR-10

In [8]:

##############################################################################################
# CIFAR-10 classification WITHOUT differential privacy: Model, Optimizer, DataLoader
##############################################################################################

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_cifar = models.resnet18(num_classes=10)

model_cifar = model_cifar.to(device)

optimizer = optim.Adam(model_cifar.parameters(), lr=LR)

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model_cifar, train_loader, optimizer, epoch + 1, device)
    if epoch % 5 == 0:
        eval(model_cifar, val_loader, device)

top1_acc_cifar = test(model_cifar, test_loader, device)
print("CIFAR Top1 Acc: ", top1_acc_cifar)
torch.save(model_cifar, 'cifar-10.pt')

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.702427 Acc@1: 38.242188 
	Train Epoch: 1 	Loss: 1.546938 Acc@1: 43.906250 
	Train Epoch: 1 	Loss: 1.448125 Acc@1: 47.653646 


Epoch:   5%|▌         | 1/20 [00:08<02:50,  8.98s/epoch]

	Validation set: Loss: 1.224714 Acc: 55.839844 
	Train Epoch: 2 	Loss: 1.079440 Acc@1: 61.390625 
	Train Epoch: 2 	Loss: 1.066495 Acc@1: 61.898438 
	Train Epoch: 2 	Loss: 1.044719 Acc@1: 62.747396 


Epoch:  10%|█         | 2/20 [00:16<02:30,  8.39s/epoch]

...

	Train Epoch: 19 	Loss: 0.049687 Acc@1: 98.265625 
	Train Epoch: 19 	Loss: 0.051065 Acc@1: 98.324219 
	Train Epoch: 19 	Loss: 0.056738 Acc@1: 98.135417 


Epoch:  95%|█████████▌| 19/20 [02:33<00:07,  7.95s/epoch]

	Train Epoch: 20 	Loss: 0.096324 Acc@1: 96.468750 
	Train Epoch: 20 	Loss: 0.092657 Acc@1: 96.738281 
	Train Epoch: 20 	Loss: 0.082905 Acc@1: 97.070312 


Epoch: 100%|██████████| 20/20 [02:41<00:00,  8.06s/epoch]


	Test set: Loss: 1.339175 Acc: 72.714844 
CIFAR Top1 Acc:  0.7271484375


# Task 1b)

In [7]:
from medmnist import BloodMNIST
from medmnist import DermaMNIST
from medmnist import OrganCMNIST

In [8]:
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_dataset_blood = BloodMNIST(split='train', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
test_dataset_blood = BloodMNIST(split='test', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
val_dataset_blood = BloodMNIST(split='val', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))

train_dataset_derma = DermaMNIST(split='train', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
test_dataset_derma = DermaMNIST(split='test', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
val_dataset_derma = DermaMNIST(split='val', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))

train_dataset_organc = OrganCMNIST(split='train', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
test_dataset_organc = OrganCMNIST(split='test', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))
val_dataset_organc = OrganCMNIST(split='val', download=True, transform=data_transform, target_transform=transforms.Lambda(lambda y: y[0]))

Using downloaded and verified file: /home/vietp/.medmnist/bloodmnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/bloodmnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/bloodmnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/dermamnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/organcmnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/organcmnist.npz
Using downloaded and verified file: /home/vietp/.medmnist/organcmnist.npz


## With differential privacy

### BloodMNist

In [12]:
##############################################################################################
# BloodMNist image classification WITH differential privacy
##############################################################################################

train_loader_blood_dp = DataLoader(train_dataset_blood, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_blood_dp = DataLoader(test_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_blood_dp = DataLoader(val_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_blood_dp = models.resnet18(num_classes=8)
model_blood_dp = ModuleValidator.fix(model_blood_dp)
ModuleValidator.validate(model_blood_dp, strict=False)

model_blood_dp = model_blood_dp.to(device)

optimizer_blood_dp = optim.Adam(model_blood_dp.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_blood_dp)}, Optimizer:{type(optimizer_blood_dp)}, DataLoader:{type(train_loader_blood_dp)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_blood_dp, optimizer_blood_dp, train_loader_blood_dp = privacy_engine.make_private_with_epsilon(
    module=model_blood_dp,
    optimizer=optimizer_blood_dp,
    data_loader=train_loader_blood_dp,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_blood_dp.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_blood_dp)}, Optimizer:{type(optimizer_blood_dp)}, DataLoader:{type(train_loader_blood_dp)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_blood_dp, train_loader_blood_dp, optimizer_blood_dp, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_blood_dp, val_loader_blood_dp, device)

top1_acc_blood_dp = test_dp(model_blood_dp, test_loader_blood_dp, device)
print("BloodMNIST DP Top1 Acc: ", top1_acc_blood_dp)
torch.save(model_blood_dp, 'bloodmnist_dp.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3993988037109375 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.549079 Acc@1: 43.742945 (ε = 12.16, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:25<08:11, 25.89s/epoch]

	Train Epoch: 2 	Loss: 0.819487 Acc@1: 71.174798 (ε = 16.51, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [08:39<00:25, 25.67s/epoch]

	Train Epoch: 20 	Loss: 0.530652 Acc@1: 87.705923 (ε = 49.30, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [09:05<00:00, 27.29s/epoch]

	Validation set: Loss: 0.665176 Acc: 86.429586 


	Test set: Loss: 0.687833 Acc: 85.245956 
BloodMNIST DP Top1 Acc:  0.8524595574116743


### DermaMNist

In [16]:
##############################################################################################
# DermaMNist image classification WITH differential privacy
##############################################################################################

train_loader_derma_dp = DataLoader(train_dataset_derma, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_derma_dp = DataLoader(test_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_derma_dp = DataLoader(val_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_derma_dp = models.resnet18(num_classes=7)
model_derma_dp = ModuleValidator.fix(model_derma_dp)
ModuleValidator.validate(model_derma_dp, strict=False)

model_derma_dp = model_derma_dp.to(device)

optimizer_derma_dp = optim.Adam(model_derma_dp.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_derma_dp)}, Optimizer:{type(optimizer_derma_dp)}, DataLoader:{type(train_loader_derma_dp)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_derma_dp, optimizer_derma_dp, train_loader_derma_dp = privacy_engine.make_private_with_epsilon(
    module=model_derma_dp,
    optimizer=optimizer_derma_dp,
    data_loader=train_loader_derma_dp,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_derma_dp.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_derma_dp)}, Optimizer:{type(optimizer_derma_dp)}, DataLoader:{type(train_loader_derma_dp)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_derma_dp, train_loader_derma_dp, optimizer_derma_dp, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_derma_dp, val_loader_derma_dp, device)

top1_acc_derma_dp = test_dp(model_derma_dp, test_loader_derma_dp, device)
print("DermaMNIST DP Top1 Acc: ", top1_acc_derma_dp)
torch.save(model_derma_dp, 'dermamnist_dp.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.42240142822265625 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.138626 Acc@1: 64.600440 (ε = 13.45, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:44<13:56, 44.04s/epoch]

	Train Epoch: 2 	Loss: 1.831852 Acc@1: 67.221568 (ε = 17.42, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [15:32<00:27, 27.86s/epoch]

	Train Epoch: 20 	Loss: 1.195714 Acc@1: 74.188244 (ε = 49.75, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [15:55<00:00, 47.77s/epoch]

	Validation set: Loss: 1.181695 Acc: 72.080286 


	Test set: Loss: 1.210802 Acc: 71.458700 
DermaMNIST DP Top1 Acc:  0.714587001173709


### OrganCMNist

In [30]:
##############################################################################################
# OrganCMNist image classification WITH differential privacy
##############################################################################################

train_loader_organc_dp = DataLoader(train_dataset_organc, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_organc_dp = DataLoader(test_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_organc_dp = DataLoader(val_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_organc_dp = models.resnet18(num_classes=11)
model_organc_dp.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

model_organc_dp = ModuleValidator.fix(model_organc_dp)
ModuleValidator.validate(model_organc_dp, strict=False)

model_organc_dp = model_organc_dp.to(device)

optimizer_organc_dp = optim.Adam(model_organc_dp.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_organc_dp)}, Optimizer:{type(optimizer_organc_dp)}, DataLoader:{type(train_loader_organc_dp)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_organc_dp, optimizer_organc_dp, train_loader_organc_dp = privacy_engine.make_private_with_epsilon(
    module=model_organc_dp,
    optimizer=optimizer_organc_dp,
    data_loader=train_loader_organc_dp,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_organc_dp.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_organc_dp)}, Optimizer:{type(optimizer_organc_dp)}, DataLoader:{type(train_loader_organc_dp)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_organc_dp, train_loader_organc_dp, optimizer_organc_dp, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_organc_dp, val_loader_organc_dp, device)

top1_acc_organc_dp = test_dp(model_organc_dp, test_loader_organc_dp, device)
print("OrganCMNIST DP Top1 Acc: ", top1_acc_organc_dp)
torch.save(model_organc_dp, 'organcmnist_dp.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3960609436035156 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.884247 Acc@1: 37.626651 (ε = 11.97, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.561748 Acc@1: 49.655508 (ε = 14.32, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:25<08:06, 25.63s/epoch]

	Train Epoch: 2 	Loss: 0.886663 Acc@1: 73.358412 (ε = 16.43, δ = 1e-05)
	Train Epoch: 2 	Loss: 0.844598 Acc@1: 74.604571 (ε = 18.01, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [09:00<00:30, 30.30s/epoch]

	Train Epoch: 20 	Loss: 0.457720 Acc@1: 89.134749 (ε = 49.25, δ = 1e-05)
	Train Epoch: 20 	Loss: 0.453955 Acc@1: 89.035333 (ε = 49.89, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [09:29<00:00, 28.47s/epoch]

	Validation set: Loss: 0.294469 Acc: 93.796165 


	Test set: Loss: 1.038518 Acc: 78.592255 
OrganCMNIST DP Top1 Acc:  0.78592254784689


## Without differential privacy

### BloodMNist

In [25]:
##############################################################################################
# BloodMNist image classification WITHOUT differential privacy: Model, Optimizer, DataLoader
##############################################################################################

train_loader_blood = DataLoader(train_dataset_blood, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_blood = DataLoader(test_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_blood = DataLoader(val_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_blood = models.resnet18(num_classes=8)

model_blood = model_blood.to(device)

optimizer_blood = optim.Adam(model_blood.parameters(), lr=LR)

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model_blood, train_loader_blood, optimizer_blood, epoch + 1, device)
    if epoch % 5 == 0:
        eval(model_blood, val_loader_blood, device)

top1_acc_blood = test(model_blood, test_loader_blood, device)
print("BloodMNIST Top1 Acc: ", top1_acc_blood)
torch.save(model_blood, 'bloodmnist.pt')

Epoch:   5%|▌         | 1/20 [00:02<00:51,  2.70s/epoch]

	Validation set: Loss: 0.757498 Acc: 72.909903 


Epoch:  30%|███       | 6/20 [00:14<00:33,  2.37s/epoch]

	Validation set: Loss: 0.397067 Acc: 87.205763 


Epoch:  55%|█████▌    | 11/20 [00:26<00:22,  2.49s/epoch]

	Validation set: Loss: 0.414685 Acc: 88.139205 


Epoch:  80%|████████  | 16/20 [00:38<00:10,  2.52s/epoch]

	Validation set: Loss: 0.432387 Acc: 89.311080 


Epoch: 100%|██████████| 20/20 [00:48<00:00,  2.44s/epoch]


	Test set: Loss: 0.387899 Acc: 90.547595 
BloodMNIST Top1 Acc:  0.9054759504608294


### DermaMNist

In [26]:
##############################################################################################
# DermaMNist image classification WITHOUT differential privacy: Model, Optimizer, DataLoader
##############################################################################################

train_loader_derma = DataLoader(train_dataset_derma, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_derma = DataLoader(test_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_derma = DataLoader(val_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_derma = models.resnet18(num_classes=7)

model_derma = model_derma.to(device)

optimizer_derma = optim.Adam(model_derma.parameters(), lr=LR)

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model_derma, train_loader_derma, optimizer_derma, epoch + 1, device)
    if epoch % 5 == 0:
        eval(model_derma, val_loader_derma, device)

top1_acc_derma = test(model_derma, test_loader_derma, device)
print("DermaMNIST Top1 Acc: ", top1_acc_derma)
torch.save(model_derma, 'dermamnist.pt')

Epoch:   5%|▌         | 1/20 [00:01<00:32,  1.71s/epoch]

	Validation set: Loss: 1.003484 Acc: 65.601729 


Epoch:  30%|███       | 6/20 [00:09<00:22,  1.61s/epoch]

	Validation set: Loss: 0.843521 Acc: 69.028424 


Epoch:  55%|█████▌    | 11/20 [00:17<00:14,  1.58s/epoch]

	Validation set: Loss: 0.932363 Acc: 74.947224 


Epoch:  80%|████████  | 16/20 [00:24<00:06,  1.55s/epoch]

	Validation set: Loss: 0.939870 Acc: 76.828873 


Epoch: 100%|██████████| 20/20 [00:30<00:00,  1.54s/epoch]


	Test set: Loss: 1.210837 Acc: 73.851737 
DermaMNIST Top1 Acc:  0.7385173672241784


### OrganCMNist

In [34]:
##############################################################################################
# OrganCMNist image classification WITHOUT differential privacy: Model, Optimizer, DataLoader
##############################################################################################

train_loader_organc = DataLoader(train_dataset_organc, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_organc = DataLoader(test_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_organc = DataLoader(val_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_organc = models.resnet18(num_classes=11)
model_organc.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

model_organc = model_organc.to(device)

optimizer_organc = optim.Adam(model_organc.parameters(), lr=LR)

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model_organc, train_loader_organc, optimizer_organc, epoch + 1, device)
    if epoch % 5 == 0:
        eval(model_organc, val_loader_organc, device)

top1_acc_organc = test(model_organc, test_loader_organc, device)
print("OrganCMNIST Top1 Acc: ", top1_acc_organc)
torch.save(model_organc, 'organcmnist.pt')

Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 0.508720 Acc@1: 82.578125 


Epoch:   5%|▌         | 1/20 [00:02<00:54,  2.85s/epoch]

	Validation set: Loss: 0.323220 Acc: 92.475142 


Epoch:  10%|█         | 2/20 [00:05<00:47,  2.64s/epoch]

...

Epoch:  95%|█████████▌| 19/20 [00:49<00:02,  2.63s/epoch]

	Train Epoch: 19 	Loss: 0.011830 Acc@1: 99.601562 


Epoch: 100%|██████████| 20/20 [00:52<00:00,  2.60s/epoch]

	Train Epoch: 20 	Loss: 0.017090 Acc@1: 99.414062 


	Test set: Loss: 0.665782 Acc: 87.830816 
OrganCMNIST Top1 Acc:  0.8783081638755981


# Task 1c)

## Differential Privacy Experiments

### Ex. 01 - Max Grad Norm

In [35]:
##############################################################################################
# Ex.01 - Norm: BloodMNist image classification WITH differential privacy
##############################################################################################

MAX_GRAD_NORM_EXPERIMENTAL = 10

train_loader_blood_dp_ex_01 = DataLoader(train_dataset_blood, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_blood_dp_ex_01 = DataLoader(test_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_blood_dp_ex_01 = DataLoader(val_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_blood_dp_ex_01 = models.resnet18(num_classes=8)
model_blood_dp_ex_01 = ModuleValidator.fix(model_blood_dp_ex_01)
ModuleValidator.validate(model_blood_dp_ex_01, strict=False)

model_blood_dp_ex_01 = model_blood_dp_ex_01.to(device)

optimizer_blood_dp_ex_01 = optim.Adam(model_blood_dp_ex_01.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_blood_dp_ex_01)}, Optimizer:{type(optimizer_blood_dp_ex_01)}, DataLoader:{type(train_loader_blood_dp_ex_01)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_blood_dp_ex_01, optimizer_blood_dp_ex_01, train_loader_blood_dp_ex_01 = privacy_engine.make_private_with_epsilon(
    module=model_blood_dp_ex_01,
    optimizer=optimizer_blood_dp_ex_01,
    data_loader=train_loader_blood_dp_ex_01,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM_EXPERIMENTAL,
)
print(f"Using sigma={optimizer_blood_dp_ex_01.noise_multiplier} and C={MAX_GRAD_NORM_EXPERIMENTAL}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_blood_dp_ex_01)}, Optimizer:{type(optimizer_blood_dp_ex_01)}, DataLoader:{type(train_loader_blood_dp_ex_01)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_blood_dp_ex_01, train_loader_blood_dp_ex_01, optimizer_blood_dp_ex_01, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_blood_dp_ex_01, val_loader_blood_dp_ex_01, device)

top1_acc_blood_dp_ex_01 = test_dp(model_blood_dp_ex_01, test_loader_blood_dp_ex_01, device)
print("BloodMNIST DP Top1 Acc: ", top1_acc_blood_dp_ex_01)
torch.save(model_blood_dp_ex_01, 'bloodmnist_dp_ex_01.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3993988037109375 and C=10
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.473115 Acc@1: 46.104333 (ε = 12.03, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:23<07:27, 23.55s/epoch]

	Train Epoch: 2 	Loss: 0.739440 Acc@1: 72.771180 (ε = 16.51, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [07:47<00:25, 25.49s/epoch]

	Train Epoch: 20 	Loss: 0.456381 Acc@1: 87.274954 (ε = 49.30, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [08:11<00:00, 24.56s/epoch]

	Validation set: Loss: 0.484808 Acc: 87.210836 


	Test set: Loss: 0.531555 Acc: 85.511173 
BloodMNIST DP Top1 Acc:  0.8551117271505376


### Ex. 02 - Max Grad Norm

In [7]:
##############################################################################################
# Ex.01 - Norm: BloodMNist image classification WITH differential privacy
##############################################################################################

MAX_GRAD_NORM_EXPERIMENTAL = 5

train_loader_blood_dp_ex_02 = DataLoader(train_dataset_blood, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_blood_dp_ex_02 = DataLoader(test_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_blood_dp_ex_02 = DataLoader(val_dataset_blood, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_blood_dp_ex_02 = models.resnet18(num_classes=8)
model_blood_dp_ex_02 = ModuleValidator.fix(model_blood_dp_ex_02)
ModuleValidator.validate(model_blood_dp_ex_02, strict=False)

model_blood_dp_ex_02 = model_blood_dp_ex_02.to(device)

optimizer_blood_dp_ex_02 = optim.Adam(model_blood_dp_ex_02.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_blood_dp_ex_02)}, Optimizer:{type(optimizer_blood_dp_ex_02)}, DataLoader:{type(train_loader_blood_dp_ex_02)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_blood_dp_ex_02, optimizer_blood_dp_ex_02, train_loader_blood_dp_ex_02 = privacy_engine.make_private_with_epsilon(
    module=model_blood_dp_ex_02,
    optimizer=optimizer_blood_dp_ex_02,
    data_loader=train_loader_blood_dp_ex_02,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM_EXPERIMENTAL,
)
print(f"Using sigma={optimizer_blood_dp_ex_02.noise_multiplier} and C={MAX_GRAD_NORM_EXPERIMENTAL}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_blood_dp_ex_02)}, Optimizer:{type(optimizer_blood_dp_ex_02)}, DataLoader:{type(train_loader_blood_dp_ex_02)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_blood_dp_ex_02, train_loader_blood_dp_ex_02, optimizer_blood_dp_ex_02, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_blood_dp_ex_02, val_loader_blood_dp_ex_02, device)

top1_acc_blood_dp_ex_02 = test_dp(model_blood_dp_ex_02, test_loader_blood_dp_ex_02, device)
print("BloodMNIST DP Top1 Acc: ", top1_acc_blood_dp_ex_02)
torch.save(model_blood_dp_ex_02, 'bloodmnist_dp_ex_02.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3993988037109375 and C=5
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.438818 Acc@1: 49.552707 (ε = 12.16, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:25<07:56, 25.09s/epoch]

	Train Epoch: 1 	Loss: 1.160884 Acc@1: 58.614613 (ε = 14.52, δ = 1e-05)
	Train Epoch: 2 	Loss: 0.791681 Acc@1: 73.647692 (ε = 16.51, δ = 1e-05)
	Train Epoch: 2 	Loss: 0.791049 Acc@1: 74.300289 (ε = 18.14, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [07:47<00:25, 25.54s/epoch]

	Train Epoch: 20 	Loss: 0.429433 Acc@1: 88.650244 (ε = 49.30, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [08:12<00:00, 24.64s/epoch]

	Validation set: Loss: 0.523246 Acc: 86.769481 


	Test set: Loss: 0.515319 Acc: 86.125012 
BloodMNIST DP Top1 Acc:  0.8612501200076804


### Ex. 03 - Epsilon

In [9]:
##############################################################################################
# Ex. 03 - Epsilon: DermaMNist image classification WITH differential privacy
##############################################################################################

EPSILON_EXPERIMENTAL = 5.0

train_loader_derma_dp_ex_03 = DataLoader(train_dataset_derma, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_derma_dp_ex_03 = DataLoader(test_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_derma_dp_ex_03 = DataLoader(val_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_derma_dp_ex_03 = models.resnet18(num_classes=7)
model_derma_dp_ex_03 = ModuleValidator.fix(model_derma_dp_ex_03)
ModuleValidator.validate(model_derma_dp_ex_03, strict=False)

model_derma_dp_ex_03 = model_derma_dp_ex_03.to(device)

optimizer_derma_dp_ex_03 = optim.Adam(model_derma_dp_ex_03.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_derma_dp_ex_03)}, Optimizer:{type(optimizer_derma_dp_ex_03)}, DataLoader:{type(train_loader_derma_dp_ex_03)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_derma_dp_ex_03, optimizer_derma_dp_ex_03, train_loader_derma_dp_ex_03 = privacy_engine.make_private_with_epsilon(
    module=model_derma_dp_ex_03,
    optimizer=optimizer_derma_dp_ex_03,
    data_loader=train_loader_derma_dp_ex_03,
    epochs=EPOCHS,
    target_epsilon=EPSILON_EXPERIMENTAL,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_derma_dp_ex_03.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_derma_dp_ex_03)}, Optimizer:{type(optimizer_derma_dp_ex_03)}, DataLoader:{type(train_loader_derma_dp_ex_03)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_derma_dp_ex_03, train_loader_derma_dp_ex_03, optimizer_derma_dp_ex_03, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_derma_dp_ex_03, val_loader_derma_dp_ex_03, device)

top1_acc_derma_dp_ex_03 = test_dp(model_derma_dp_ex_03, test_loader_derma_dp_ex_03, device)
print("DermaMNIST DP Top1 Acc: ", top1_acc_derma_dp_ex_03)
torch.save(model_derma_dp_ex_03, 'dermamnist_dp_ex_03.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=1.05224609375 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.201841 Acc@1: 64.484611 (ε = 1.39, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:13<04:15, 13.45s/epoch]

	Train Epoch: 2 	Loss: 2.026563 Acc@1: 66.285088 (ε = 1.77, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [04:22<00:14, 14.08s/epoch]

	Train Epoch: 20 	Loss: 1.346760 Acc@1: 70.922473 (ε = 4.97, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [04:36<00:00, 13.84s/epoch]

	Validation set: Loss: 1.269244 Acc: 69.427776 


	Test set: Loss: 1.284995 Acc: 69.906057 
DermaMNIST DP Top1 Acc:  0.6990605743838028


### Ex. 04 - Epsilon

In [10]:
##############################################################################################
# Ex. 04 - Epsilon: DermaMNist image classification WITH differential privacy
##############################################################################################

EPSILON_EXPERIMENTAL = 25.0

train_loader_derma_dp_ex_04 = DataLoader(train_dataset_derma, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_derma_dp_ex_04 = DataLoader(test_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_derma_dp_ex_04 = DataLoader(val_dataset_derma, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_derma_dp_ex_04 = models.resnet18(num_classes=7)
model_derma_dp_ex_04 = ModuleValidator.fix(model_derma_dp_ex_04)
ModuleValidator.validate(model_derma_dp_ex_04, strict=False)

model_derma_dp_ex_04 = model_derma_dp_ex_04.to(device)

optimizer_derma_dp_ex_04 = optim.Adam(model_derma_dp_ex_04.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_derma_dp_ex_04)}, Optimizer:{type(optimizer_derma_dp_ex_04)}, DataLoader:{type(train_loader_derma_dp_ex_04)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_derma_dp_ex_04, optimizer_derma_dp_ex_04, train_loader_derma_dp_ex_04 = privacy_engine.make_private_with_epsilon(
    module=model_derma_dp_ex_04,
    optimizer=optimizer_derma_dp_ex_04,
    data_loader=train_loader_derma_dp_ex_04,
    epochs=EPOCHS,
    target_epsilon=EPSILON_EXPERIMENTAL,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_derma_dp_ex_04.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_derma_dp_ex_04)}, Optimizer:{type(optimizer_derma_dp_ex_04)}, DataLoader:{type(train_loader_derma_dp_ex_04)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_derma_dp_ex_04, train_loader_derma_dp_ex_04, optimizer_derma_dp_ex_04, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_derma_dp_ex_04, val_loader_derma_dp_ex_04, device)

top1_acc_derma_dp_ex_04 = test_dp(model_derma_dp_ex_04, test_loader_derma_dp_ex_04, device)
print("DermaMNIST DP Top1 Acc: ", top1_acc_derma_dp_ex_04)
torch.save(model_derma_dp_ex_04, 'dermamnist_dp_ex_04.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.538330078125 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 2.110428 Acc@1: 65.391363 (ε = 7.46, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:19<06:06, 19.31s/epoch]

	Train Epoch: 2 	Loss: 1.940049 Acc@1: 66.494617 (ε = 9.42, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [05:47<00:18, 18.10s/epoch]

	Train Epoch: 20 	Loss: 1.185503 Acc@1: 74.068895 (ε = 24.90, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [06:04<00:00, 18.21s/epoch]

	Validation set: Loss: 1.153002 Acc: 71.823886 


	Test set: Loss: 1.232268 Acc: 70.736136 
DermaMNIST DP Top1 Acc:  0.7073613556338028


### Ex. 05 - Delta

In [9]:
##############################################################################################
# Ex. 05 - Delta: OrganCMNist image classification WITH differential privacy
##############################################################################################

DELTA_EXPERIMENTAL = 1e-2

train_loader_organc_dp_ex_05 = DataLoader(train_dataset_organc, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_organc_dp_ex_05 = DataLoader(test_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_organc_dp_ex_05 = DataLoader(val_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_organc_dp_ex_05 = models.resnet18(num_classes=11)
model_organc_dp_ex_05.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

model_organc_dp_ex_05 = ModuleValidator.fix(model_organc_dp_ex_05)
ModuleValidator.validate(model_organc_dp_ex_05, strict=False)

model_organc_dp_ex_05 = model_organc_dp_ex_05.to(device)

optimizer_organc_dp_ex_05 = optim.Adam(model_organc_dp_ex_05.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_organc_dp_ex_05)}, Optimizer:{type(optimizer_organc_dp_ex_05)}, DataLoader:{type(train_loader_organc_dp_ex_05)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_organc_dp_ex_05, optimizer_organc_dp_ex_05, train_loader_organc_dp_ex_05 = privacy_engine.make_private_with_epsilon(
    module=model_organc_dp_ex_05,
    optimizer=optimizer_organc_dp_ex_05,
    data_loader=train_loader_organc_dp_ex_05,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA_EXPERIMENTAL,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_organc_dp_ex_05.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_organc_dp_ex_05)}, Optimizer:{type(optimizer_organc_dp_ex_05)}, DataLoader:{type(train_loader_organc_dp_ex_05)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_organc_dp_ex_05, train_loader_organc_dp_ex_05, optimizer_organc_dp_ex_05, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_organc_dp_ex_05, val_loader_organc_dp_ex_05, device)

top1_acc_organc_dp_ex_05 = test_dp(model_organc_dp_ex_05, test_loader_organc_dp_ex_05, device)
print("OrganCMNIST DP Top1 Acc: ", top1_acc_organc_dp_ex_05)
torch.save(model_organc_dp_ex_05, 'organcmnist_dp_ex_05.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3408622741699219 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.867050 Acc@1: 38.015462 (ε = 17.29, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.527441 Acc@1: 50.910245 (ε = 21.15, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:28<08:58, 28.35s/epoch]

	Train Epoch: 2 	Loss: 0.880425 Acc@1: 73.805611 (ε = 24.38, δ = 1e-05)
	Train Epoch: 2 	Loss: 0.822548 Acc@1: 75.900426 (ε = 26.92, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [08:51<00:29, 29.04s/epoch]

	Train Epoch: 20 	Loss: 0.425848 Acc@1: 89.663649 (ε = 78.75, δ = 1e-05)
	Train Epoch: 20 	Loss: 0.409707 Acc@1: 89.783272 (ε = 79.83, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [09:21<00:00, 28.09s/epoch]

	Validation set: Loss: 0.358763 Acc: 92.684659 


	Test set: Loss: 0.998275 Acc: 79.737341 
OrganCMNIST DP Top1 Acc:  0.7973734051036683


### Ex. 06 - Delta

In [14]:
##############################################################################################
# Ex. 06 - Delta: OrganCMNist image classification WITH differential privacy
##############################################################################################

DELTA_EXPERIMENTAL = 1e-1

train_loader_organc_dp_ex_06 = DataLoader(train_dataset_organc, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader_organc_dp_ex_06 = DataLoader(test_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
val_loader_organc_dp_ex_06 = DataLoader(val_dataset_organc, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model_organc_dp_ex_06 = models.resnet18(num_classes=11)
model_organc_dp_ex_06.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

model_organc_dp_ex_06 = ModuleValidator.fix(model_organc_dp_ex_06)
ModuleValidator.validate(model_organc_dp_ex_06, strict=False)

model_organc_dp_ex_06 = model_organc_dp_ex_06.to(device)

optimizer_organc_dp_ex_06 = optim.Adam(model_organc_dp_ex_06.parameters(), lr=LR)

print(
    f"Before make_private(). "
    f"Model:{type(model_organc_dp_ex_06)}, Optimizer:{type(optimizer_organc_dp_ex_06)}, DataLoader:{type(train_loader_organc_dp_ex_06)}"
)

# Now we initialize a privacy engine and attach it
privacy_engine = PrivacyEngine()
model_organc_dp_ex_06, optimizer_organc_dp_ex_06, train_loader_organc_dp_ex_06 = privacy_engine.make_private_with_epsilon(
    module=model_organc_dp_ex_06,
    optimizer=optimizer_organc_dp_ex_06,
    data_loader=train_loader_organc_dp_ex_06,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA_EXPERIMENTAL,
    max_grad_norm=MAX_GRAD_NORM,
)
print(f"Using sigma={optimizer_organc_dp_ex_06.noise_multiplier} and C={MAX_GRAD_NORM}")

print("="*20)

print(
    f"After make_private(). "
    f"Model:{type(model_organc_dp_ex_06)}, Optimizer:{type(optimizer_organc_dp_ex_06)}, DataLoader:{type(train_loader_organc_dp_ex_06)}"
)
# Now it's business as usual

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train_dp(model_organc_dp_ex_06, train_loader_organc_dp_ex_06, optimizer_organc_dp_ex_06, epoch + 1, device, privacy_engine)
    if (epoch + 1) % 5 == 0:
        eval_dp(model_organc_dp_ex_06, val_loader_organc_dp_ex_06, device)

top1_acc_organc_dp_ex_06 = test_dp(model_organc_dp_ex_06, test_loader_organc_dp_ex_06, device)
print("OrganCMNIST DP Top1 Acc: ", top1_acc_organc_dp_ex_06)
torch.save(model_organc_dp_ex_06, 'organcmnist_dp_ex_06.pt')

Before make_private(). Model:<class 'torchvision.models.resnet.ResNet'>, Optimizer:<class 'torch.optim.adam.Adam'>, DataLoader:<class 'torch.utils.data.dataloader.DataLoader'>
Using sigma=0.3116035461425781 and C=1.2
After make_private(). Model:<class 'opacus.grad_sample.grad_sample_module.GradSampleModule'>, Optimizer:<class 'opacus.optimizers.optimizer.DPOptimizer'>, DataLoader:<class 'opacus.data_loader.DPDataLoader'>


Epoch:   0%|          | 0/20 [00:00<?, ?epoch/s]

	Train Epoch: 1 	Loss: 1.805972 Acc@1: 40.140469 (ε = 21.46, δ = 1e-05)
	Train Epoch: 1 	Loss: 1.493614 Acc@1: 52.651670 (ε = 26.33, δ = 1e-05)


Epoch:   5%|▌         | 1/20 [00:25<08:06, 25.59s/epoch]

	Train Epoch: 2 	Loss: 0.842316 Acc@1: 75.059347 (ε = 30.89, δ = 1e-05)
	Train Epoch: 2 	Loss: 0.812366 Acc@1: 76.569741 (ε = 34.09, δ = 1e-05)


...

Epoch:  95%|█████████▌| 19/20 [08:55<00:29, 29.51s/epoch]

	Train Epoch: 20 	Loss: 0.378154 Acc@1: 90.813869 (ε = 103.83, δ = 1e-05)
	Train Epoch: 20 	Loss: 0.383092 Acc@1: 90.711985 (ε = 105.31, δ = 1e-05)


Epoch: 100%|██████████| 20/20 [09:25<00:00, 28.27s/epoch]

	Validation set: Loss: 0.293334 Acc: 93.845881 


	Test set: Loss: 1.011871 Acc: 79.728618 
OrganCMNIST DP Top1 Acc:  0.7972861842105263
